In [1]:
from datasets import load_dataset
from transformers import AutoProcessor, WhisperModel
import torch
import wave

# Step 1: Load the Audio Data
path_to_audio = "data/sub/De95Osq7p1c_trimmed_segment_1.wav"

/home/dhuser/Desktop/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-20 10:05:56.658096: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-20 10:05:56.691374: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 10:05:57.233026: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import wave
import numpy as np

def read_wav_file(file_path):
    # Open the WAV file
    with wave.open(file_path, 'rb') as wav_file:
        # Get the number of frames in the file
        n_frames = wav_file.getnframes()
        
        # Read the frame data
        frame_data = wav_file.readframes(n_frames)
        
        # Get the sample width (in bytes)
        sample_width = wav_file.getsampwidth()
        
        # Get the number of channels
        n_channels = wav_file.getnchannels()
        
        # Get the frame rate (samples per second)
        frame_rate = wav_file.getframerate()
        
        # Convert the byte data to a numpy array
        if sample_width == 1:
            # 8-bit audio
            audio_data = np.frombuffer(frame_data, dtype=np.uint8)
        elif sample_width == 2:
            # 16-bit audio
            audio_data = np.frombuffer(frame_data, dtype=np.int16)
        elif sample_width == 4:
            # 32-bit audio
            audio_data = np.frombuffer(frame_data, dtype=np.int32)
        else:
            raise ValueError("Unsupported sample width: {}".format(sample_width))
        
        # Reshape the array based on the number of channels
        if n_channels > 1:
            audio_data = audio_data.reshape(-1, n_channels)
        
        return audio_data, frame_rate

In [3]:
audio_data, frame_rate = read_wav_file(path_to_audio)

print("Audio Data Shape:", audio_data.shape)
print("Frame Rate:", frame_rate)

Audio Data Shape: (54864,)
Frame Rate: 16000


In [4]:
processor = AutoProcessor.from_pretrained("openai/whisper-medium")

/home/dhuser/Desktop/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
inputs = processor(
    audio_data,
    return_tensors="pt",
    return_attention_mask=True,
    sampling_rate=16000     
)

In [6]:
inputs

{'input_features': tensor([[[1.4301, 1.4301, 1.4301,  ..., 1.4301, 1.4301, 1.4301],
         [1.4301, 1.4301, 1.4301,  ..., 1.4301, 1.4301, 1.4301],
         [1.4301, 1.4301, 1.4301,  ..., 1.4301, 1.4301, 1.4301],
         ...,
         [1.4301, 1.4301, 1.4301,  ..., 1.4301, 1.4301, 1.4301],
         [1.4301, 1.4301, 1.4301,  ..., 1.4301, 1.4301, 1.4301],
         [1.4301, 1.4301, 1.4301,  ..., 1.4301, 1.4301, 1.4301]]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}

In [7]:
# Process the Audio with Whisper Encoder
model = WhisperModel.from_pretrained("openai/whisper-medium")
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = inputs.to(device)

# Ensuring No Gradient Updates
with torch.no_grad():
    encoder_outputs = model.encoder(**inputs, output_hidden_states=True)


In [8]:
# Step 3: Retrieve the Hidden States
hidden_states = encoder_outputs.hidden_states
audio_embeddings = encoder_outputs.last_hidden_state

print("Shape of audio embeddings:", audio_embeddings.shape)

Shape of audio embeddings: torch.Size([1, 1500, 1024])


In [9]:
from transformers import WhisperForConditionalGeneration

model_generate = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")
model_generate.to(device)

generated = model_generate.generate(inputs = inputs.input_features)
processor.batch_decode(generated)

/home/dhuser/Desktop/.venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:473: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


["<|startoftranscript|><|en|><|transcribe|><|notimestamps|> I'm gonna make a new friend!<|endoftext|>"]

## Projecting Audio Embeddings

In [ ]:
class Adaptor(nn.Module):
    def __init__(self, output_embedding_size = 4096):
        super().__init__()
        self.pool1 = torch.nn.AdaptiveAvgPool1d(256)
        self.linear = torch.nn.Linear(1280, output_embedding_size, bias=False)
        self.ln1 = torch.nn.LayerNorm(1280)
    
